In [ ]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import time

In [ ]:
# Load the IMDB dataset
max_features = 5000
maxlen = 100
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

17464789/17464789 [==============================] - 1s 0us/step


In [ ]:
# Preprocess the data
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

In [ ]:
# Convert sequences back into text
reverse_word_index = dict([(value, key) for (key, value) in imdb.get_word_index().items()])
def decode_review(text):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in text])

X_train_text = [decode_review(x) for x in X_train]
X_test_text = [decode_review(x) for x in X_test]

# Create and fit tokenizer
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train_text)

# Preprocess the data
X_train = pad_sequences(tokenizer.texts_to_sequences(X_train_text), maxlen=maxlen)
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test_text), maxlen=maxlen)

1641221/1641221 [==============================] - 1s 0us/step


In [ ]:
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
start_time = time.time()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=32)
training_time = time.time() - start_time

Epoch 1/5
782/782 [==============================] - 165s 205ms/step - loss: 0.4239 - accuracy: 0.8007 - val_loss: 0.4054 - val_accuracy: 0.8119
Epoch 2/5
782/782 [==============================] - 160s 205ms/step - loss: 0.2989 - accuracy: 0.8752 - val_loss: 0.3410 - val_accuracy: 0.8484
Epoch 3/5
782/782 [==============================] - 159s 204ms/step - loss: 0.2485 - accuracy: 0.9002 - val_loss: 0.3585 - val_accuracy: 0.8462
Epoch 4/5
782/782 [==============================] - 158s 202ms/step - loss: 0.2106 - accuracy: 0.9148 - val_loss: 0.4069 - val_accuracy: 0.8459
Epoch 5/5
782/782 [==============================] - 157s 201ms/step - loss: 0.1752 - accuracy: 0.9310 - val_loss: 0.4548 - val_accuracy: 0.8432


In [ ]:
# Evaluate the model
start_time = time.time()

score, acc = model.evaluate(X_test, y_test, batch_size=32)
prediction_time = time.time() - start_time

print("Test accuracy:", acc)

782/782 [==============================] - 21s 27ms/step - loss: 0.4548 - accuracy: 0.8432
Test accuracy: 0.8432000279426575


In [ ]:
# Function to preprocess user input
def preprocess_input(user_input):
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    sequences = tokenizer.texts_to_sequences(user_input)
    return pad_sequences(sequences, maxlen=maxlen)

#from tensorflow.keras.preprocessing.text import Tokenizer
    #tokenizer = Tokenizer(num_words=max_features)
    #tokenizer.fit_on_texts(user_input)
    #sequences = tokenizer.texts_to_sequences(user_input)
    #return pad_sequences(sequences, maxlen=maxlen)

In [ ]:
# Function to predict sentiment
def predict_sentiment(review):
    review = preprocess_input([review])
    prediction = model.predict(review)[0][0]
    if prediction >= 0.5:
        return "Positive"
    else:
        return "Negative"

In [ ]:
# User Interface
print("Welcome to Movie Review Sentiment Analyzer!")

Welcome to Movie Review Sentiment Analyzer!


In [ ]:
while True:
    user_review = input("Enter your movie review (or type 'quit' to exit): ")
    if user_review.lower() == 'quit':
        print("Thank you for using our service!")
        break
    else:
        sentiment = predict_sentiment(user_review)
        print("Predicted sentiment:", sentiment)

Enter your movie review (or type 'quit' to exit): good
1/1 [==============================] - 0s 379ms/step
Predicted sentiment: Positive
Enter your movie review (or type 'quit' to exit): bad
1/1 [==============================] - 0s 29ms/step
Predicted sentiment: Negative
Enter your movie review (or type 'quit' to exit): love it
1/1 [==============================] - 0s 28ms/step
Predicted sentiment: Positive
Enter your movie review (or type 'quit' to exit): hate it
1/1 [==============================] - 0s 35ms/step
Predicted sentiment: Negative
Enter your movie review (or type 'quit' to exit): quit
Thank you for using our service!
